## Importing Libraries and dataset

In [1]:
# !pip install pingouin
# !pip install xlwt

In [2]:
import pandas as pd
import numpy as np

from scipy.stats import f_oneway
import pingouin as pg

import xlwt
from xlwt import Workbook

In [3]:
# wb = Workbook()

# sheet = wb.add_sheet('Anova')

In [4]:
video_id = 9

In [5]:
delta = 2
theta = 3
alpha1 = 4
alpha2 = 5
beta1 = 6
beta2 = 7
gamma1 = 8
gamma2 = 9

In [6]:
video_col = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [7]:
df1 = pd.read_csv("./EEG_data.csv")

In [8]:
df1.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0


In [9]:
# df1 = df1.loc[df1['VideoID'] == video_id]

In [10]:
df1.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0


In [11]:
df2 = pd.read_csv("./demographic_info.csv")

In [12]:
df2.head()

,subject ID,age,ethnicity,gender
0,0,25,Han Chinese,M
1,1,24,Han Chinese,M
2,2,31,English,M
3,3,28,Han Chinese,F
4,4,24,Bengali,M


## Preprocessing

### Rename column 'subject ID' in df2 to 'SubjectID' so that we can perform inner join.

In [13]:
df2 = df2.rename(columns = {'subject ID': 'SubjectID'})

### Perform Inner join

In [14]:
df = pd.merge(df1, df2, on='SubjectID', how='inner')

In [15]:
df.shape

(12811, 18)

In [16]:
df.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln,age,ethnicity,gender
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0,25,Han Chinese,M
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0,25,Han Chinese,M
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0,25,Han Chinese,M
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0,25,Han Chinese,M
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0,25,Han Chinese,M


In [17]:
# 'Attention', 'Mediation', 'Delta', 'Theta', 'Alpha1', 'Beta1', 'Beta2', 'Gamma1', 'Gamma2'

feature = 'Delta'

class0 = []
class1 = []

In [18]:
# row_no = video_col[video_id]
# col_no = gamma2

In [19]:
values = df[feature].values
outs = df['user-definedlabeln'].values

for i in range(len(values)):
    if outs[i] == 0:
        class0.append(values[i])
    elif outs[i] == 1:
        class1.append(values[i])
        
print(len(class0))
print(len(class1))

6244
6567


In [20]:
# f_oneway(class0, class1)

In [21]:
# ans = f_oneway(class0, class1)[1]
# ans

In [22]:
pg.ttest(class0, class1, correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-17.050717,12770.312758,two-sided,1.782075e-64,"[-211329.13, -167750.18]",0.300585,4.539e+60,1.0


In [23]:
ans = pg.ttest(class0, class1, correction=True)['p-val'][0]
ans

1.7820754987434584e-64

In [24]:
# sheet.write(row_no, col_no, ans)

In [25]:
# wb.save('ttest_video wise.xls')